In [ ]:
import os
from collections import namedtuple
import math
import json

import numpy as np
import pandas as pd
from skimage.feature import peak_local_max
from scipy.stats import multivariate_normal
from scipy.optimize import leastsq
import progressbar
import matplotlib.pyplot as plt
import imageio

from utoolbox.container import Volume

### Environment presets

In [ ]:
file_path = os.path.join(
    *["data", "20170831_SIM", "SI_b1a2DSIM_os", "decWF", "RAWb1a2DSIM_os_ch0_stack0000_561nm.tif"]
)

In [ ]:
dx = 50
dy = 50
dz = 300

In [ ]:
kernel_size = 15
radius = (kernel_size-1) // 2

In [ ]:
results = pd.DataFrame(
    columns=[
        'px', 'py', 'pz', # bounding box center
        'a',              # amplitude
        'cx', 'cy', 'cz', # center
        'wx', 'wy', 'wz', # sigma
    ]
)

### Load data

In [ ]:
raw = Volume(file_path)
print("shape={}".format(raw.shape))
print("dtype={}".format(raw.dtype))

In [ ]:
print("mean={:.2f}, sd={:.2f}".format(np.mean(raw), np.std(raw)))

### Find peaks

In [ ]:
th = np.mean(raw) + 2*np.std(raw)
coords = peak_local_max(
    raw, exclude_border=radius, min_distance=2*kernel_size, threshold_abs=th
)

In [ ]:
print("{} peaks found".format(len(coords)))

### Isolate the patches

In [ ]:
patches = np.zeros((len(coords), kernel_size, kernel_size, kernel_size), dtype=np.float32)

In [ ]:
index = 0
bar = progressbar.ProgressBar()
for coord in bar(coords):
    x = coord[2]
    y = coord[1]
    z = coord[0]
    
    results.loc[index, 'px'] = x
    results.loc[index, 'py'] = y
    results.loc[index, 'pz'] = z
    
    patches[index, ...] = raw[z-radius:z+radius+1, y-radius:y+radius+1, x-radius:x+radius+1]
    
    index += 1

### Preview selected beads

In [ ]:
raw_xy = np.amax(raw, axis=0)

In [ ]:
plt.figure(figsize=(10, 10))
plt.autoscale(enable=True, tight=True)

plt.imshow(raw_xy, cmap='jet')
plt.scatter(coords[:, 2], coords[:, 1], s=100, marker='o', facecolors='none', edgecolor='w')

### 3D fitting

In [ ]:
Params = namedtuple('Params', 
    ['a',              # amplitude
     'cx', 'cy', 'cz', # centroid
     'wx', 'wy', 'wz'  # width
    ]
)

The fitting function.

In [ ]:
def gaussian_3d(params):
    params = Params(*params)
    return lambda x, y, z: \
        params.a * np.exp(
            -( ((params.cx-x)/params.wx)**2 + 
               ((params.cy-y)/params.wz)**2 +
               ((params.cz-z)/params.wz)**2
             ) / 2
        )

In [ ]:
def moments(data):
    s = data.sum()
    
    zi, yi, xi = np.indices(data.shape)
    cx = (xi*data).sum() / s
    cy = (yi*data).sum() / s
    cz = (zi*data).sum() / s
    
    t = data[int(cz), int(cy), :]
    wx = np.sqrt(np.abs((np.arange(t.size)-cx)**2 * t).sum() / t.sum())
    t = data[int(cz), :, int(cx)]
    wy = np.sqrt(np.abs((np.arange(t.size)-cy)**2 * t).sum() / t.sum())
    t = data[:, int(cy), int(cx)]
    wz = np.sqrt(np.abs((np.arange(t.size)-cz)**2 * t).sum() / t.sum())
    
    a = data.max()
    
    return Params(a, cx, cy, cz, wx, wy, wz)

In [ ]:
def fit_gaussian_3d(data):
    """Returns fitting result of a 3D Gaussian distribution."""
    params = moments(data)
    err_func = lambda params: \
        np.ravel(gaussian_3d(params)(*np.indices(data.shape)) - data)
    optparams, success = leastsq(err_func, params)
    return Params(*optparams), success

In [ ]:
index = 0
n_failed = 0
bar = progressbar.ProgressBar()
for patch in bar(patches):
    optparams, is_success = fit_gaussian_3d(patch)
    
    results.loc[index, 'a'] = optparams.a
    
    results.loc[index, 'cx'] = optparams.cx
    results.loc[index, 'cy'] = optparams.cy
    results.loc[index, 'cz'] = optparams.cz
    
    results.loc[index, 'wx'] = optparams.wx
    results.loc[index, 'wy'] = optparams.wy
    results.loc[index, 'wz'] = optparams.wz
    
    if not is_success:
        n_failed += 1
        
    index += 1

print("failed to fit {} beads".format(n_failed))

### Statistics

In [ ]:
wx = results['wx'].mean()
wx_e = results['wx'].std()

wy = results['wy'].mean()
wy_e = results['wy'].std()

wz = results['wz'].mean()
wz_e = results['wz'].std()

$\text{FWHM} = 2 \sqrt{2 ln2} \sigma$

In [ ]:
factor = 2 * math.sqrt(2 * math.log(2))

In [ ]:
xres = wx * factor * dx
yres = wy * factor * dy
zres = wz * factor * dz

In [ ]:
print("FWHM resolution (x, y, z) = ({:.4f}, {:.4f}, {:.4f}) nm".format(xres, yres, zres))

### Save the result to files

Save cropped patches.

In [ ]:
patches_folder = os.path.join(os.path.dirname(file_path), "patches")
if not os.path.exists(patches_folder):
    os.makedirs(patches_folder)
    
index = 0
bar = progressbar.ProgressBar()
for patch in bar(patches):
    patches_path = os.path.join(patches_folder, "{}.tif".format(index))
    imageio.volwrite(patches_path, patch)
    index += 1

Save fitting results.

In [ ]:
results_path = os.path.join(os.path.dirname(file_path), "fitting_results.csv")
results.to_csv(results_path, index=False)

Statistics summary.

In [ ]:
summary_path = os.path.join(os.path.dirname(file_path), "summary.json")
with open(summary_path, 'w') as outfile:
    json.dump(
        {
            'source': os.path.basename(file_path),
            'unit': 'nm',
            'factor': factor,
            'spacing': {
                'x': dx,
                'y': dy, 
                'z': dz
            },
            'resolution': {
                'x': xres,
                'y': yres,
                'z': zres
            }
        }, 
        outfile, 
        indent=4, 
    )

### Fitting test

In [ ]:
i_patch = 40
data = patches[i_patch, ...]

optparams, success = fit_gaussian_3d(data)
print("success? {}".format(success))

Params(*optparams)

In [ ]:
plt.figure(figsize=(5, 5))
plt.autoscale(enable=True, tight=True)

data_fit = gaussian_3d(optparams)(*np.indices(data.shape))
plt.imshow(np.amax(data, axis=0), cmap='jet')
plt.contour(np.amax(data_fit, axis=0), cmap='jet')

In [ ]:
results